In [13]:
# load data files
# PLEASE MODIFY THE PATH OF DATA!!!!!!!!!!!!!!!!!!!!!!!!!
# ===========================================================================================
# 下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下

# data_subfold = "data_2023_03_01-16_48_29"    # the data from Lukas
# data_subfold = "rosbag2_2023_06_05-16_57_44"
data_subfold = "rosbag2_2023_06_05-17_22_10"
# data_subfold = "rosbag2_2023_06_05-17_26_41"

save_merged_data = True

execfile("data_process_from_rosbags.py")
# 上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上
# ===========================================================================================


# Files
from model import *
from model_kf import *
from model_all import *

# Packages
import pandas as pd
import numpy as np

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
output_notebook()

def plot_config(width: int, height: int,title=None,
                x_label: str = None, 
                y_label: str = None):
    p = figure(title=title, background_fill_color="#fafafa", width=width, height=height)
    p.output_backend = "svg"    # save plot in HTML as SVG
    p.title.text_font_size = "15pt"
    p.xaxis.axis_label = x_label
    p.yaxis.axis_label = y_label
    p.yaxis.axis_label_text_font_size = "12pt"
    p.yaxis.major_label_text_font_size = "12pt"
    p.xaxis.major_label_text_font_size = "12pt"
    p.xaxis.axis_label_text_font_size = "12pt"
    return p

./data/rosbag2_2023_06_05-17_22_10/data_merge.csv is saved

Defined Variables:
    df_command
    df_imu
    df_velocity
    df_tf
    df_merge    # merged from 'df_command', 'df_velocity', 'df_imu' and 'df_tf'

Finish!


Loading BokehJS ...

In [88]:
df = pd.read_csv(r'./data/' + data_subfold + '/data_merge.csv')
data = df.to_numpy()

for i, item in enumerate(df.columns):
    print(i, item)
df.head(20)


0 __time
1 command.linear.x
2 command.linear.y
3 command.linear.z
4 command.angular.x
5 command.angular.y
6 command.angular.z
7 velocity.v
8 imu.header.stamp
9 imu.w_x
10 imu.w_y
11 imu.w_z
12 imu.a_x
13 imu.a_y
14 imu.a_z
15 tf.header.stamp
16 tf.x
17 tf.y
18 tf.z
19 tf.r_x
20 tf.r_y
21 tf.r_z
22 tf.r_w
23 tf.roll_x
24 tf.pitch_y
25 tf.yaw_z


,__time,command.linear.x,command.linear.y,command.linear.z,command.angular.x,command.angular.y,command.angular.z,velocity.v,imu.header.stamp,imu.w_x,...,tf.x,tf.y,tf.z,tf.r_x,tf.r_y,tf.r_z,tf.r_w,tf.roll_x,tf.pitch_y,tf.yaw_z
0,1.685979e+09,NaN,NaN,NaN,NaN,NaN,NaN,-0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.685979e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.685979e+09,-0.006981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.685979e+09,NaN,NaN,NaN,NaN,NaN,NaN,-0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.685979e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.685979e+09,-0.008727,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.685979e+09,NaN,NaN,NaN,NaN,NaN,NaN,-0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.685979e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.685979e+09,-0.006981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.685979e+09,NaN,NaN,NaN,NaN,NaN,NaN,-0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.685979e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.685979e+09,-0.001745,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.685979e+09,NaN,NaN,NaN,NaN,NaN,NaN,-0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.685979e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.685979e+09,-0.001745,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df_imu
# df_imu["imu.header.stamp"].to_numpy().shape
# df_imu["imu.w_x"].to_numpy().shape

,__time,imu.header.stamp,imu.w_x,imu.w_y,imu.w_z,imu.a_x,imu.a_y,imu.a_z
0,1.685979e+09,1.685979e+09,-0.006981,-0.001745,-0.001745,0.068647,-9.728196,0.568786
1,1.685979e+09,1.685979e+09,-0.008727,-0.003491,-0.001745,0.068647,-9.728196,0.568786
2,1.685979e+09,1.685979e+09,-0.006981,0.000000,0.000000,0.068647,-9.728196,0.578592
3,1.685979e+09,1.685979e+09,-0.001745,-0.006981,0.000000,0.068647,-9.728196,0.578592
4,1.685979e+09,1.685979e+09,-0.001745,-0.001745,0.000000,0.058840,-9.728196,0.588399
...,...,...,...,...,...,...,...,...
3749,1.685979e+09,1.685979e+09,-0.006981,-0.003491,0.000000,0.098066,-9.738003,0.490332
3750,1.685979e+09,1.685979e+09,-0.003491,0.000000,0.000000,0.098066,-9.738003,0.490332
3751,1.685979e+09,1.685979e+09,-0.003491,-0.001745,0.000000,0.098066,-9.738003,0.490332
3752,1.685979e+09,1.685979e+09,-0.005236,0.000000,0.000000,0.088260,-9.738003,0.490332


In [82]:
print(np.average(df_imu["imu.a_z"].to_numpy()))
print(np.average(df_imu["imu.a_z"].to_numpy()[:900]))


0.5503322552129444
0.572305184006691


In [84]:
t0 = data[:,0][0]

p = plot_config(width=900, height=600, title='0000000',  x_label='t [ s ]', y_label= 'unknown ')

# p.circle(x=data[:,0]-t0, y=data[:,9],legend_label='w_x', size=3, fill_color='red', line_color='red')
# p.circle(x=data[:,0]-t0, y=data[:,10],legend_label='w_y', size=3, fill_color='green', line_color='green')
# p.circle(x=data[:,0]-t0, y=data[:,11],legend_label='w_z', size=3, fill_color='blue', line_color='blue')

p.circle(x=df_imu["imu.header.stamp"].to_numpy(), y=df_imu["imu.a_x"].to_numpy(),legend_label='a_x', size = 2, fill_color='red', line_color='red')
p.circle(x=df_imu["imu.header.stamp"].to_numpy(), y=df_imu["imu.a_y"].to_numpy(),legend_label='a_y', size = 2, fill_color='green', line_color='green')
p.circle(x=df_imu["imu.header.stamp"].to_numpy(), y=df_imu["imu.a_z"].to_numpy(),legend_label='a_z', size = 2, fill_color='blue', line_color='blue')
# p.circle(x=df_imu["imu.header.stamp"].to_numpy()[:900], y=df_imu["imu.a_z"].to_numpy()[:900],legend_label='a_z', size = 2, fill_color='blue', line_color='blue')


# p.line(x=df_imu["imu.header.stamp"].to_numpy(), y=df_imu["imu.w_x"].to_numpy(),legend_label='w_x', line_width = 2,  line_color='red')
# p.line(x=df_imu["imu.header.stamp"].to_numpy(), y=df_imu["imu.w_y"].to_numpy(),legend_label='w_y', line_width = 2, line_color='green')
# p.line(x=df_imu["imu.header.stamp"].to_numpy(), y=df_imu["imu.w_z"].to_numpy(),legend_label='w_z', line_width = 2, line_color='blue')
# [:900]

# p.line(x=df_imu["imu.header.stamp"].to_numpy(), y=df_imu["imu.a_x"].to_numpy(),legend_label='a_x', line_width = 2,  line_color='red')
# p.line(x=df_imu["imu.header.stamp"].to_numpy(), y=df_imu["imu.a_y"].to_numpy(),legend_label='a_y', line_width = 2, line_color='green')
# p.line(x=df_imu["imu.header.stamp"].to_numpy()[:900], y=df_imu["imu.a_z"].to_numpy()[:900],legend_label='a_z', line_width = 2, line_color='blue')

# output_file(filename="./w_y VS yaw.html", title="KF_all vs tf")
show(p, notebook_handle=True)

In [43]:
df_imu["imu.header.stamp"].to_numpy()

array([1.68597853e+09, 1.68597853e+09, 1.68597853e+09, ...,
       1.68597855e+09, 1.68597855e+09, 1.68597855e+09])

In [85]:
# ==========================================Simulation================================================
a_z = df_imu["imu.a_z"].to_numpy().reshape(1,-1) - 0.55
w_y = df_imu["imu.w_y"].to_numpy().reshape(1,-1)
U_k = np.append(a_z, w_y, axis=0)  # 'input' of simulation system

timestamp = df_imu["imu.header.stamp"].to_numpy().reshape(1,-1) - df_imu["imu.header.stamp"].to_numpy()[0]  # global time stamp start from 0s
Ts = np.diff(timestamp)  # time steps of time stamps, number of data will be -1

# x0 = [df_tf["tf.x"][0], df_tf["tf.y"][0], df_tf["tf.yaw_z"][0], 0]  # initial states
x0 = [df_tf["tf.x"][0], df_tf["tf.y"][0], 0, 0]  # initial states
x_all = sim(x0, U_k, timestamp)
print("shape of result", x_all.shape)

# ============================================ Plot =============================================
p_total = []

# simulation curve
p = plot_config(width=800, height=800, title='imu curve', x_label='x [ unknown ]', y_label='y [ unknown ]')
p.match_aspect=True
p.aspect_scale=1
p.circle(x_all[0][0], x_all[1][0], fill_color="red", legend_label='start', size=10)
p.circle(x=x_all[0], y=x_all[1], legend_label='KF curve', size=3, line_color='blue',fill_color='blue')
p.asterisk(x_all[0][-1], x_all[1][-1], line_color="green", legend_label='end', size=10)

p.circle(df_tf["tf.x"][0], df_tf["tf.y"][0], fill_color="red", legend_label='start', size=10)
p.circle(df_tf["tf.x"], df_tf["tf.y"], legend_label='tf curve', size=2, line_color='black', fill_color='black')
p.asterisk(df_tf["tf.x"].to_numpy()[-1], df_tf["tf.y"].to_numpy()[-1], line_color="green", legend_label='end', size=10)
p_total.append(p)

# simulation states
p2 = plot_config(width=900, height=600, title='system states', x_label='t [ s ]', y_label='states [ unknown ] ')
p2.line(x=timestamp[0], y=x_all[0], legend_label='x', line_width=2, line_color='green')
p2.line(x=timestamp[0], y=x_all[1], legend_label='y', line_width=2, line_color='black')
p2.line(x=timestamp[0], y=x_all[2], legend_label='theta', line_width=2, line_color='royalblue')
p2.line(x=timestamp[0], y=x_all[3], legend_label='v', line_width=2, line_color='crimson')
p_total.append(p2)

show(p, notebook_handle=True)
show(p2, notebook_handle=True)


shape of result (4, 3754)


In [90]:
t0 = data[:,0][0]

p = plot_config(width=900, height=600, title='0000000',  x_label='t [ s ]', y_label= 'unknown ')

# p.circle(x=data[:,0]-t0, y=data[:,2]/180*np.pi,legend_label='command steering angle', size=2, fill_color='black', line_color='black')
# p.circle(x=data[:,0]-t0, y=data[:,3],legend_label='command velocity', size=2, fill_color='green', line_color='green')
# p.circle(x=data[:,0]-t0, y=data[:,3]*np.tan(data[:,2]/180*np.pi),legend_label='command w', size=2, fill_color='blue', line_color='blue')

# p.circle(x=data[:,0]-t0, y=data[:,7],legend_label='imu angular velocity w_y', size=2, fill_color='blue', line_color='blue')
# p.circle(x=data[:,5]-t0, y=data[:,11],legend_label='imu linear acceleration a_z', size=2, fill_color='blue', line_color='blue')

# p.circle(x=data[:,0]-t0, y=data[:,4]/-1000,legend_label='vel', size=2, fill_color='red', line_color='red')

p.circle(x=data[:,0]-data[:,0][0], y=data[:,-1],legend_label='tf yaw angle __time', size=3, fill_color='red', line_color='red')
p.circle(x=data[:,-11]-data[:,0][0], y=data[:,-1],legend_label='tf yaw angle header', size=2, fill_color='blue', line_color='blue')

# output_file(filename="./w_y VS yaw.html", title="KF_all vs tf")
show(p, notebook_handle=True)

In [ ]:
p = plot_config(width=900, height=600, title='0000000',  x_label='t [ s ]', y_label= 'unknown ')

p.circle(x=data[:,0]-data[:,0][-10], y=data[:,-1],legend_label='tf yaw angle __time', size=3, fill_color='red', line_color='red')
p.circle(x=data[:,-11]-data[:,-11][-11], y=data[:,-1],legend_label='tf yaw angle header', size=2, fill_color='blue', line_color='blue')

# output_file(filename="./w_y VS yaw.html", title="KF_all vs tf")
show(p, notebook_handle=True)

In [91]:
t0 = data[:,0][0]

p = plot_config(width=900, height=600, title='0000000',  x_label='t [ s ]', y_label= 'unknown ')

# p.circle(x=data[:,0]-t0, y=data[:,2]/180*np.pi,legend_label='command steering angle', size=2, fill_color='black', line_color='black')
# p.circle(x=data[:,0]-t0, y=data[:,3],legend_label='command velocity', size=2, fill_color='green', line_color='green')
# p.circle(x=data[:,0]-t0, y=data[:,3]*np.tan(data[:,2]/180*np.pi),legend_label='command w', size=2, fill_color='blue', line_color='blue')

p.circle(x=data[:,0]-t0, y=data[:,7],legend_label='imu angular velocity w_y', size=2, fill_color='blue', line_color='blue')
# p.circle(x=data[:,5]-t0, y=data[:,11],legend_label='imu linear acceleration a_z', size=2, fill_color='blue', line_color='blue')

# p.circle(x=data[:,0]-t0, y=data[:,4]/-1000,legend_label='vel', size=2, fill_color='red', line_color='red')

p.circle(x=data[:,0]-data[:,0][0], y=data[:,-1],legend_label='tf yaw angle __time', size=3, fill_color='red', line_color='red')
# p.circle(x=data[:,-11]-data[:,-11][1], y=data[:,-1],legend_label='tf yaw angle header', size=2, fill_color='blue', line_color='blue')

# output_file(filename="./w_y VS yaw.html", title="KF_all vs tf")
show(p, notebook_handle=True)